[УТВЕРЖДЕНЫ
постановлением Правительства
Российской Федерации
от 31 октября 2023 г. N 1809](https://ivo.garant.ru/#/document/407923417/paragraph/4/doclist/6925/1/0/0/%D0%9F%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F%20%D1%80%D0%B5%D0%BA%D1%80%D0%B5%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F%20%D0%B5%D0%BC%D0%BA%D0%BE%D1%81%D1%82%D1%8C%20%D0%B2%20%D0%B3%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D0%B0%D1%85%20%D1%86%D0%B5%D0%BD%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B9%20%D1%8D%D0%BA%D0%BE%D0%BB%D0%BE%D0%B3%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B9%20%D0%B7%D0%BE%D0%BD%D1%8B%20%D0%91%D0%B0%D0%B9%D0%BA%D0%B0%D0%BB%D1%8C%D1%81%D0%BA%D0%BE%D0%B9%20%D0%BF%D1%80%D0%B8%D1%80%D0%BE%D0%B4%D0%BD%D0%BE%D0%B9%20%D1%82%D0%B5%D1%80%D1%80%D0%B8%D1%82%D0%BE%D1%80%D0%B8%D0%B8:0)

# Типовые правила расчета предельно допустимой рекреационной емкости особо охраняемых природных территорий регионального и местного значения при осуществлении туризма

### 1. Настоящие Типовые правила устанавливают порядок расчета предельно допустимой рекреационной емкости особо охраняемых природных территорий регионального и местного значения в рамках организации на их территориях туризма.

Расчет предельно допустимой рекреационной емкости особо охраняемых природных территорий регионального и местного значения, расположенных в границах центральной экологической зоны Байкальской природной территории, в рамках организации на их территориях туризма исполнительными органами Республики Бурятия и Иркутской области осуществляется с учетом требований статьи 12 Федерального закона "Об охране озера Байкал".

### 2. В целях настоящих Типовых правил используются следующие понятия:

- **"автономный многодневный туристский маршрут"** - туристский маршрут, длящийся более одного дня, при котором туристы не используют места размещения (туристские базы, гостиницы, кемпинги) и пункты питания туристов;

- **"базовая рекреационная емкость"** - максимальное количество человек, которые могут физически находиться на особо охраняемой природной территории или в отдельной части (на туристском объекте) в единицу времени;

- **"лимитирующий фактор развития туризма"** - фактор, ограничивающий возможности развития туризма на особо охраняемой природной территории или в отдельной части (на туристском объекте) ввиду несовместимости туризма и обеспечения сохранности уникальных и типичных природных комплексов и объектов, объектов растительного и животного мира, естественных экологических систем, биоразнообразия в целях поддержания их в естественном состоянии, а также невозможности оказания посетителям услуг, соответствующих договору или обычно предъявляемым к ним требованиям, и снижения негативного воздействия на местную социокультурную среду;

- **"линейный туристский объект (туристский маршрут)"** - путь следования туристов, расположенный на особо охраняемой природной территории, характеризующийся линейным характером воздействия на окружающую среду вдоль пути следования;

- **"многодневный туристский маршрут"** - туристский маршрут, прохождение которого длится более одного дня, при котором туристы размещаются на ночевку в местах размещения;

- **"однодневный туристский маршрут"** - туристский маршрут, прохождение которого совершается в течение дня, без использования мест размещения;

- **"площадной туристский объект"** - туристский объект, занимающий определенный земельный участок или участок акватории, имеющий свои границы, характеризующийся свободным перемещением на нем посетителей и площадным (обширным, по всей площади объекта или его части) характером воздействия его на окружающую среду (стоянки, смотровые площадки, музеи, визит-центры, иные здания и сооружения для организации обслуживания туристов);

- **"потенциальная рекреационная емкость"** - максимальное количество человек, которые могут находиться на особо охраняемой природной территории, в ее отдельной части или на туристском объекте в единицу времени без деградации природных комплексов и объектов, объектов растительного и животного мира, естественных экологических систем, с учетом факторов экологического, социального, социокультурного и социально-экономического характера;

- **"предельно допустимая рекреационная емкость"** - максимальное количество посетителей, которые могут посетить в качестве туриста особо охраняемую природную территорию либо ее отдельные части в единицу времени без деградации природных комплексов и объектов, объектов растительного и животного мира, естественных экологических систем;

- **"туристские объекты"** - части особо охраняемой природной территории, включающие природные комплексы и объекты, историко-культурные объекты, инфраструктурные объекты (музеи, визит-центры, иные здания и сооружения для организации обслуживания туристов), привлекающие туристов и используемые для осуществления туризма.

### 3. Предельно допустимая рекреационная емкость определяется для особо охраняемой природной территории либо ее отдельных частей (туристских объектов).

### 4. Предельно допустимая рекреационная емкость устанавливается исполнительными органами субъектов Российской Федерации в отношении находящихся в их ведении особо охраняемых природных территорий регионального значения и органами местного самоуправления в отношении находящихся в их ведении особо охраняемых природных территорий местного значения.

### 5. Расчет предельно допустимой рекреационной емкости особо охраняемой природной территории, ее отдельной части осуществляется при выявлении изменения состояния туристских объектов, но не реже одного раза в 5 лет.

### 6. Предельно допустимая рекреационная емкость рассчитывается для особо охраняемой природной территории в целом, а также для ее отдельных частей (туристских объектов) и выражается в целочисленных значениях, человек в единицу времени.



In [52]:
# conda activate //anaconda3/envs/condageoenv
# Или base и оставь ядро gdal
import math
import folium
import os
import zipfile
import gpxpy
import geojson
import geopandas as gpd
import osmnx as ox
import requests
from pyproj import CRS

from shapely.geometry import mapping, shape
from pyproj import Proj, transform

from shapely import wkt
from shapely.geometry import Polygon, MultiPolygon, shape

from shapely.wkt import loads

from shapely.geometry import Polygon, MultiPolygon, shape
from shapely import wkt
# import psycopg2
# from psycopg2 import sql
# from shapely.geometry import shape
# from shapely.ops import transform
# import pyproj
# from functools import partial
# import json

In [53]:
# Конвертор

def files_from_folder(name_folder: str, exp: str) -> list: 
    '''
    Выгрузить все файлы заданного расширения из папки в список
    name_folder - относительный путь к папке с файлами
    exp - фильтр расширений имени файлов
    '''
    
    base_dir = os.getcwd()
    path = os.path.join(base_dir, name_folder)
    names = os.listdir(path)
    path_zip_file = [os.path.join(path, name) for name in names if name.endswith(exp)]
    
    return path_zip_file


def add_gpx_to_map(zip_path: str) -> geojson:
    '''
    Импортировать и отобразить маршруты из .gpx файла
    zip_path - относительный путь к .gpx файлу
    '''
    with zipfile.ZipFile(zip_path, 'r') as z:
        for filename in z.namelist():
            if filename.endswith('.gpx'):

                # Читаем содержимое .gpx файла
                with z.open(filename) as f:
                    gpx_data = gpxpy.parse(f)

                    geojson_features = []

                    for track in gpx_data.tracks:
                        for segment in track.segments:
                            points = [(point.longitude, point.latitude) for point in segment.points]
                            # Формируем geojson
                            line = geojson.LineString(points)
                            feature = geojson.Feature(geometry=line, properties={"popup": f"Маршрут по GPS из файла {filename}"})
                            geojson_features.append(feature)

                    feature_collection = geojson.FeatureCollection(geojson_features)

                    return feature_collection

In [84]:
def project_geometry(geometry, target_crs):
    '''
    Функция для преобразования геометрии в другую проекцию с использованием GeoPandas.
    '''
    # Создаем GeoSeries из геометрии
    gdf = gpd.GeoSeries([geometry])
    
    # Устанавливаем систему координат исходных данных, если необходимо
    gdf.set_crs(epsg=4326, inplace=True)  # Предполагается, что исходная проекция WGS84 (EPSG:4326)

    # Проводим преобразование в заданную систему координат
    gdf = gdf.to_crs(target_crs)
    
    # Возвращаем преобразованную геометрию
    return gdf.geometry[0]


In [55]:
def add_features_to_layers_from_gpx(my_map):
    '''
    Добавление слоя на карту по данным .gpx
    '''
    
    feature_gps_routes = folium.FeatureGroup(name="Маршруты GPS", show=True)
    feature_gps_routes.add_to(my_map)
    
    # Выгружаем из заданной папки корневой директории файлы заданного расширения 
    list_zip_files = files_from_folder('../data', '.zip')

    # Добавляем линейные маршруты из файлов .gpx на карту
    for zip_file in list_zip_files:
        gpx_geojson = add_gpx_to_map(zip_file)

        # Проходим по всем элементам в FeatureCollection
        for feature in gpx_geojson['features']:
            popup_content = f"{feature['properties']['popup']}"

            # Добавляем GeoJSON на карту
            folium.GeoJson(
                feature,  # Добавляем отдельный Feature
                popup=folium.Popup(popup_content, max_width=300),
                style_function=lambda x: {
                    'color': 'red',
                    'weight': 2,
                    'opacity': 0.7
                },
            ).add_to(feature_gps_routes)

    return my_map

In [56]:
# Генератор

def fetch_features_with_generator(administrative_name: str, wiki_tags: dict) -> dict:
    '''
    Генератор для извлечения объектов из Overpass API
    administrative_name - административное название места для выгрузки объектов
    wiki_tags - выгружаемые теги согласно https://wiki.openstreetmap.org/wiki/
    '''
    try:
        features = ox.features_from_place(administrative_name, tags=wiki_tags)
        for _, row in features.iterrows():
            if row.geometry is not None: 
                yield row
            else:
                print(f"Warning: строка без геометрии: {row}")
    except requests.exceptions.Timeout as e:
        print(f"Overpass API timeout: {e}. Попробуйте уменьшить область или разделить запрос.")
        return 
    except Exception as e:
        print(f"Error!: {e}")

In [99]:
def convert_data_to_shapely(data):
    '''
    Создание ограничивающего полигона.
    Конвертирует данные из словаря и геометрии WKT в формат геометрии Shapely.  
    Обрабатывает возможные MultiPolygon внутри.
    Args:
    data: Словарь с геометрическими данными.
    Returns:
    Shapely геометрию (Polygon, MultiPolygon или None) в случае ошибки.
    '''
    shapely_geometries = []
    for item in data.values():
        try:
            geom_wkt = item['geometry']
            shapely_geom = wkt.loads(geom_wkt)
            shapely_geometries.append(shapely_geom)
        except Exception as e:
            print(f"Ошибка обработки геометрии: {e}")
            return None

    if len(shapely_geometries) == 1:
        return shapely_geometries[0]
    elif len(shapely_geometries) > 1:
        all_polygons = []
        for geom in shapely_geometries:
            if isinstance(geom, MultiPolygon):
                all_polygons.extend(geom.geoms)  # Извлекаем все Polygons из MultiPolygon
            elif isinstance(geom, Polygon):
                all_polygons.append(geom)
            else:
                print(f"Неподдерживаемый тип геометрии: {type(geom)}")
                return None
        return MultiPolygon(all_polygons)  # Создаём MultiPolygon из всех Polygons

    else:
        return None


# data = {
#     'Объект: 1': {'geometry': 'POLYGON ((158.424 53.364, 158.403 53.346, 158.397 53.348, 158.396 53.351, 158.424 53.364))', 'area': 8114971737.3719845, 'length': None},
#     'Объект: 2': {'geometry': 'MULTIPOLYGON (((165.733 55.516, 165.735 55.517, 165.764 55.529, 165.794 55.541, 165.733 55.516)))', 'area': 37663228797.86329, 'length': None}
# }

# shapely_geometry = convert_data_to_shapely(data)

# print(shapely_geometry)


In [115]:
def add_features_to_layers(
        my_map, 
        place, 
        wiki_tags, 
        bounding_polygon=None, # Параметр для ограничивающего полигона
        name_layer_points=None,
        color_points=f'rgb({45}, {210}, {100})',
        name_layer_lines=None,
        color_lines=f'rgb({210}, {45}, {45})',
        name_layer_polygons=None,
        fillcolor_polygons=f'rgb({50}, {170}, {235})',
        color_polygons=f'rgb({50}, {120}, {235})'
    ) -> folium.Map:
    
    '''
    Добавление слоев на карту по wiki-тегам с группировкой их по трем группам:
    layer_points - точечные слои
    layer_lines - линейные слои
    layer_polygons - полигональные слои
    my_map - карта
    place - административное название области, например, "Камчатский край, Россия"
    wiki_tags - теги рассматриваемых объектов (согласно https://wiki.openstreetmap.org/wiki/)
    '''

    # Инициализация слоев, только если имя слоя задано
    layer_points = folium.FeatureGroup(name=name_layer_points) if name_layer_points else None
    layer_lines = folium.FeatureGroup(name=name_layer_lines) if name_layer_lines else None
    layer_polygons = folium.FeatureGroup(name=name_layer_polygons) if name_layer_polygons else None

    # Добавляем слои на карту, только если они инициализированы
    if layer_points:
        layer_points.add_to(my_map)
    if layer_lines:
        layer_lines.add_to(my_map)
    if layer_polygons:
        layer_polygons.add_to(my_map)

    # Инициализация словаря для хранения площадей и длины
    feature_lengths_areas = {}
    index = 1 

    for row in fetch_features_with_generator(place, wiki_tags):

        # Проверка на пересечение с ограничивающим полигоном (если задан)
        if bounding_polygon is not None and not row.geometry.intersects(bounding_polygon):
            continue

        # Преобразуем геометрию в Меркатор
        target_crs = CRS.from_epsg(3857)  # Проекция Меркатора
        geom_merc = project_geometry(row.geometry, target_crs)

        popup_content = f"<b>Теги:</b>{''.join([f'<li>{k}: {row.get(k)}</li>' for k in wiki_tags.keys() if row.get(k)])}<br>"

        if row.geometry.geom_type in ['Point', 'MultiPoint'] and layer_points:
            folium.GeoJson(
                row.geometry,
                popup=folium.Popup(popup_content, max_width=300),
                style_function=lambda x: {'color': color_points, 'weight': 2}
            ).add_to(layer_points)

            feature_lengths_areas[f"Объект: {index}"] = {'geometry': str(row.geometry), 'area': None, 'length': None}

        elif row.geometry.geom_type in ['LineString', 'MultiLineString'] and layer_lines:
            folium.GeoJson(
                row.geometry,
                popup=folium.Popup(popup_content, max_width=300),
                style_function=lambda x: {'color': color_lines, 'weight': 2}
            ).add_to(layer_lines)

            feature_lengths_areas[f"Объект: {index}"] = {'geometry': str(row.geometry), 'area': None, 'length': geom_merc.length}


        elif row.geometry.geom_type in ['Polygon', 'MultiPolygon'] and layer_polygons:
            folium.GeoJson(
                row.geometry,
                popup=folium.Popup(popup_content, max_width=300),
                style_function=lambda x: {'fillColor': fillcolor_polygons, 'color': color_polygons, 'weight': 2, 'fillOpacity': 0.5}
            ).add_to(layer_polygons)

            feature_lengths_areas[f"Объект: {index}"] = {'geometry': str(row.geometry), 'area': geom_merc.area, 'length': None}

        index += 1
        
    # Выводим словарь с длиной и площадью
    # print(feature_lengths_areas)

    return feature_lengths_areas

In [117]:
# Указываем название области
place_name = "Камчатский край, Россия"

# Устанавливаем URL для Overpass API
ox.settings.overpass_url = "https://overpass-api.de/api/"

# Получаем границы исследуемой территории
place_extent = ox.geocode_to_gdf(place_name)
map_center = [place_extent.geometry.centroid.y, place_extent.geometry.centroid.x]  # Центр исследуемой территории

# Создаем карту с фокусом на загруженную область
first_map = folium.Map(location=map_center, zoom_start=6)
 
# - - - - - - - - - - - - -

# Задаем теги природоохранной территории (https://wiki.openstreetmap.org/wiki/RU:Tag:leisure%3Dnature_reserve), 
# для которой будет рассчитываться придельно допустимая рекреационная емкость

# tags_oopt = {
#         'boundary': 'protected_area'
#     }

# add_features_to_layers(
#         first_map, 
#         place_name, 
#         tags_oopt, 
#         name_layer_polygons='Федеральные ООПТ'
#     )

# - - - - - - - - - - - - -

tags_parks = {
        'boundary': 'national_park'
    }

data_parks = add_features_to_layers(
        first_map, 
        place_name, 
        tags_parks, 
        name_layer_polygons='Национальные парки',
        fillcolor_polygons=f'rgb({225}, {180}, {65})', 
        color_polygons=f'rgb({181}, {142}, {35})'
    )
# Из словаря ограничивающих полигонов выбираем первый, потом второй и так далее
# в зависимости от количества оопт (в данном случае Национальных парков)
key_value_park = [{key: value} for key, value in data_parks.items()][0]
print(key_value_park)

# Конвертируем геометрию WKT Национального парка в Shapely для здания ограничивающего полигона
# (так как нам нужны не все объекты, а только в границах ООПТ (или Национальных парков, как в данном случае))
shapely_geometry_park = convert_data_to_shapely(key_value_park)

# - - - - - - - - - - - - -

# Задаем теги природоохранной территории, для которой будет рассчитываться придельно допустимая рекреационная емкость
# tags_sub_oopt = {
#         'leisure': 'nature_reserve'
#     }

# add_features_to_layers(
#         first_map, 
#         place_name, 
#         tags_sub_oopt, 
#         name_layer_polygons='Региональные и местные природные территории',
#         fillcolor_polygons=f'rgb({35}, {165}, {150})',
#         color_polygons=f'rgb({35}, {165}, {80})'
#     )

# - - - - - - - - - - - - -

# Задаем теги туристских объектов
tags = {
        'tourism': True,
        'leisure': [ # https://wiki.openstreetmap.org/wiki/RU:Key:leisure
            'adult_gaming_centre',
            'amusement_arcade',
            'bandstand',
            'bathing_place',
            'beach_resort',
            'bird_hide',
            'bleachers',
            'bowling_alley',
            'common',
            'dance',
            'disc_golf_course',
            'dog_park',
            'escape_game',
            'firepit',
            'fishing',
            'fitness_centre',
            'fitness_station',
            'garden',
            'golf_course',
            'hackerspace',
            'horse_riding',
            'ice_rink',
            'marina',
            'miniature_golf',
            'outdoor_seating',
            'park',
            'picnic_table',
            'pitch',
            'playground',
            'resort',
            'sauna',
            'slipway',
            'sports_centre',
            'sports_hall',
            'stadium',
            'summer_camp',
            'swimming_area',
            'swimming_pool',
            'tanning_salon',
            'track',
            'trampoline_park',
            'water_park',
            'wildlife_hide'
            ],
        'landuse': 'recreation_ground',
        'historic': True
    }

data_polygon = add_features_to_layers(
        first_map, 
        place_name, 
        tags, 
        shapely_geometry_park, # Ограничивающий полигон (Национальные парки)
        # name_layer_points='Точечные объекты', 
        # color_points=f'rgb({45}, {210}, {100})',
        # name_layer_lines='Линейные объекты', 
        # color_lines=f'rgb({210}, {45}, {45})',
        name_layer_polygons='Площадные объекты',
        fillcolor_polygons=f'rgb({45}, {210}, {105})',
        color_polygons=f'rgb({35}, {165}, {80})'
    )

# print(data_polygon)

# - - - - - - - - - - - - -

# Добавляем GPS-маршруты
add_features_to_layers_from_gpx(first_map)

# - - - - - - - - - - - - -

# Кнопка управления слоями
folium.LayerControl().add_to(first_map)

# Отображаем карту и сохраняем в HTML файл
first_map.save("map.html")


/var/folders/b9/wz3thh3n6176vcms9jxkj4wr0000gn/T/ipykernel_61635/1058078436.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_center = [place_extent.geometry.centroid.y, place_extent.geometry.centroid.x]  # Центр исследуемой территории
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 417 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


{'Объект: 1': {'geometry': 'POLYGON ((158.4242254 53.3638187, 158.4026336 53.3455445, 158.3973164 53.3476671, 158.3958886 53.3508654, 158.4007357 53.3540964, 158.4002758 53.3557435, 158.3918062 53.3560957, 158.3820824 53.3578048, 158.3655813 53.3648948, 158.3600369 53.3649736, 158.3568687 53.3626104, 158.3473641 53.365525, 158.3471001 53.3729288, 158.3507963 53.3780477, 158.3329751 53.3839533, 158.3269027 53.3900156, 158.3500334 53.4131455, 158.4140676 53.4154822, 158.4192253 53.4193857, 158.4208094 53.4263082, 158.4202814 53.4330722, 158.4266178 53.4374762, 158.4379706 53.4459682, 158.4453631 53.4509997, 158.4567158 53.4527291, 158.4635803 53.455559, 158.4691247 53.4591746, 158.4841737 53.470334, 158.4582999 53.4880886, 158.4519635 53.490759, 158.4503794 53.4935864, 158.4374425 53.492801, 158.4160571 53.4956282, 158.4113048 53.4968846, 158.3986319 53.5003397, 158.3880712 53.5045797, 158.4047043 53.5130582, 158.3920315 53.5227908, 158.3891273 53.5264008, 158.3901834 53.5317366, 158.391

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 417 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


### 7. Предельно допустимая рекреационная емкость особо охраняемой природной территории (RСCоопт) рассчитывается с помощью функции:

In [7]:
def calculation_RCCoopt(features: list[int], RCCqm: list[int]) -> tuple[dict, int]:
    '''
    RCCqm - предельно допустимая рекреационная емкость туристского объекта m, человек в единицу времени;
    features - список туристских объектов (0...m-1).
    '''

    # Создаем словарь с ключами из списка номеров (или названий) объектов и рассчитанными значениями RCCq
    dict_features_RCCqm = {feature: rccq for feature, rccq in zip(features, RCCqm)}
    
    result = int(sum(dict_features_RCCqm.values()))
    return dict_features_RCCqm, result

### 8. Предельно допустимая рекреационная емкость туристского объекта (RCCq) рассчитывается с помощью функции:

In [8]:
def calculation_RCCq(PCCq: list[int], MC: list[float]) -> list[int]:
    '''
    PCCq - потенциальная рекреационная емкость туристского объекта, человек в единицу времени;
    MC - коэффициент управленческой емкости, долей от единицы.
    '''
    
    result = []
    
    for pccq, mc in zip(PCCq, MC):
        result.append(int(pccq * mc))
    
    return result

### 9. Потенциальная рекреационная емкость рассчитывается для особо охраняемой природной территории в целом, а также для ее отдельных частей (туристских объектов) и выражается в целочисленных значениях, человек в единицу времени.

### 10. Потенциальная рекреационная емкость туристского объекта (PCCq) рассчитывается c помощью функции:

In [9]:
def calculation_PCCq(BCCq: list[int], Cfn: list[list[int]]) -> list[int]:
    '''
    BCCq - базовая рекреационная емкость туристского объекта, выраженная в целочисленном значении, человек в единицу времени;
    Cfn - поправочные коэффициенты, которые учитывают определенные для туристских объектов лимитирующие факторы развития туризма (экологического, социального и социокультурного характера) и установленные режимы использования туристских объектов;
    n - количество поправочных коэффициентов. 
    '''

    # Перемножаем элементы вложенных в список Cf списков
    Cf_multiplied_sublists = [math.prod(sublist) for sublist in Cfn]
    
    result = []
    
    for bccq, cf in zip(BCCq, Cf_multiplied_sublists):
        result.append(int(bccq * cf))

    return result

### 11. Базовая рекреационная емкость туристских объектов (BCC q) для площадных и линейных туристских объектов (туристских маршрутов) рассчитывается по приведенным формулам и выражается в целочисленных значениях, человек в единицу времени.

### 12. Базовая рекреационная емкость для ПЛОЩАДНЫХ туристских объектов (BCC qs) рассчитываетсяс помощь функции:

In [10]:
def calculation_BBCqs(A: list[float], Au: list[float], Rf: list[float], t: int) -> list[int]:
    '''
    A - площадь туристского объекта, на которой осуществляется туризм, кв. метров;
    Au - площадь туристского объекта, необходимая для одного посетителя при осуществлении туризма (кв. метров), определяемая в соответствии с порядком, указанным в пункте 18 настоящих Правил;
    Rf - коэффициент возвращения, отражающий возможное количество посещений туристского объекта одним и тем же туристом в день;
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц.
    '''
    
    result = []
    
    for area, area_per_visitor, return_factor in zip(A, Au, Rf):
        bbcq = (area / area_per_visitor) * return_factor * t
        result.append(int(bbcq)) 

    return result


### 13. Коэффициент возвращения (Rf) рассчитывается по формуле:

In [11]:
def calculation_Rf(T: list[int], Td: list[int]) -> list[float]:
    '''
    Т - количество часов в сутки, когда туристский объект доступен для посещения, часов;
    Td - среднее время пребывания посетителя на туристском объекте, часов.
    '''

    result = []
    
    for times, mean_times in zip(T, Td):
        rf = times / mean_times 
        result.append(float(rf)) 

    return result

### 14. Базовая рекреационная емкость для ЛИНЕЙНЫХ: однодневных туристских маршрутов и многодневных туристских маршрутов с небольшой протяженностью или несколькими входами без ограничения времени посещения туристского маршрута (BCC qp1) рассчитывается по формуле:

In [12]:
def calculation_BBCqp1(
        DTp: list[list[float]], 
        DGp: list[list[float]], 
        Ts: float,
        Tdp: list[list[float]],
        t: int,
        GS: float
        # tp: list[int]
        ) -> list[float]:
    '''
    DTp - протяженность однодневного туристского маршрута или однодневного участка p многодневного туристского маршрута в дневной переход, км;
    DGp - оптимальное расстояние между группами на участке p туристского маршрута, км;
    Ts - длина светового дня или количество времени, когда туристский маршрут доступен для посетителей, часов;
    Tdp - среднее время прохождения участка туристского маршрута p с учетом остановок, часов;
    GS - среднее количество человек в группе (включая сопровождающих), человек;
    p - порядковый номер однодневного участка туристского маршрута (1, 2, ... p);
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц;
    tp - количество дней пребывания посетителей на туристском маршруте, единиц.
    '''
    
    result = []
    p = 0 # Счетчик однодневных участков в многодневном маршруте

    # Обработка каждого вложенного списка
    for dtp_list, dgp_list, tdp_list in zip(DTp, DGp, Tdp):
        sum_for_current = 0 # хранилище сумм
        # Рассчитаем результат в отдельных вложенных списках
        for dtp, dgp, tdp in zip(dtp_list, dgp_list, tdp_list):
            # считаем результирующее значение и суммируем со следующим рассчитанным
            # в рассматриваемом вложенном списке 
            sum_for_current += (dtp / dgp) * (Ts / tdp)
        p += 1 

        tp = len(DTp[p - 1])
        result.append(round(sum_for_current * GS * (t / tp), 1)) 

    return result

# Пример входных данных (трех маршрутов 1 и 3 многодневные, а 2 - однодневный) 
# посути это список со вложенными списками разной длины
DTp = [[20, 25], [30], [30, 20, 50]]
DGp = [[5, 10], [10], [10, 5, 20]]
Ts = 8.0
Tdp = [[4, 3], [3], [3, 4, 3]]
t = 30
GS = 3.1

# Вызов функции
out = calculation_BBCqp1(
        DTp, DGp, Ts, Tdp, t, GS
        )
print(out)

[682.0, 744.0, 702.7]


### 15. Базовая рекреационная емкость для однодневных и многодневных туристских маршрутов, время доступности которых строго фиксировано (BCC qp2) (например, в случае закрытия для посетителей входа и выхода с туристского маршрута или в целом с участка особо охраняемой природной территории в четко установленные часы), рассчитывается по формуле:

In [13]:
def calculation_BBCqp2(
        gp: list[list[int]],
        t: int,
        GS: float
        ) -> list[float]:
    '''
    gp - максимальное количество групп, которые могут пройти в сутки по 
    однодневному участку p туристского маршрута до его закрытия или до окончания светового дня, единиц;
    GS - среднее количество человек в группе (включая сопровождающих), человек;
    p - порядковый номер однодневного участка туристского маршрута (1, 2, ... p);
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц;
    tp - количество дней пребывания посетителей на туристском маршруте, единиц.
    '''
    
    result = []
    p = 0 # Счетчик однодневных участков в многодневном маршруте

    # Обработка каждого вложенного списка
    for gp_list in gp:
        sum_for_current = 0 # хранилище сумм
        # Рассчитаем результат в отдельных вложенных списках
        for gp_value in gp_list:
            # считаем результирующее значение и суммируем со следующим рассчитанным
            # в рассматриваемом вложенном списке 
            sum_for_current += gp_value * GS
        p += 1 

        tp = len(gp[p - 1])
        result.append(round(sum_for_current * (t / tp), 1)) 

    return result

gp = [[5, 6], [7], [7, 5, 12]]
t = 30
GS = 3.0
# Вызов функции
out = calculation_BBCqp2(
        gp, t, GS
        )
print(out)

[495.0, 630.0, 720.0]


### 16. Максимальное количество групп, которые могут пройти в сутки по однодневному участку туристского маршрута до его закрытия или до окончания светового дня (gp), выражается целочисленным значением (единиц) и определяется по формуле:

In [14]:
def calculation_gp(
        DGp: list[list[float]], 
        Ts: float,
        Tdp: list[list[float]],
        vp: list[float]
        ) -> list[list[int]]:
    '''
    DGp - оптимальное расстояние между группами на участке p туристского маршрута, км;
    Ts - длина светового дня или количество времени, когда туристский маршрут доступен для посетителей, часов;
    Tdp - среднее время прохождения участка туристского маршрута p с учетом остановок, часов;
    vp - средняя скорость передвижения по однодневному участку p туристского маршрута с учетом остановок, км в час.
    '''
    
    result = []

    for dgp_list, tdp_list, vp_list in zip(DGp, Tdp, vp):
        subresult = []
        for dgp, tdp, vp_value in zip(dgp_list, tdp_list, vp_list):
            calculated_value = int(1 + ((vp_value * (Ts - tdp)) / dgp))
            subresult.append(calculated_value)
        result.append(subresult)

    return result

DGp = [[5, 10], [10], [10, 5, 20]]
Ts = 8.0
Tdp = [[4, 3], [3], [3, 4, 3]]
vp = [[4.0, 4.0], [5.0], [4.0, 4.0, 6.0]]

# Вызов функции
out = calculation_gp(
        DGp, Ts, Tdp, vp
        )
print(out)

[[4, 3], [3], [3, 4, 2]]


### 17. Базовая рекреационная емкость для автономных многодневных туристских маршрутов (BBCqp3) рассчитывается по формуле:

In [15]:
def calculation_BBCqp3(
        gp: list[list[int]],
        t: int,
        GS: float
        ) -> list[list[int]]:
    '''
    gp - максимальное количество групп, которые могут пройти в сутки по однодневному участку 
    туристского маршрута до его закрытия или до окончания светового дня (gp), единиц;
    gpmin - минимальное из рассчитанных для однодневных участков туристского маршрута значений 
    максимального количества групп, которые могут пройти в сутки по однодневному участку p 
    туристского маршрута до его закрытия или до окончания светового дня, единиц;
    GS - среднее количество человек в группе (включая сопровождающих), человек;
    p - порядковый номер однодневного участка туристского маршрута (1, 2, ... p);
    t - количество дней в рассматриваемую единицу времени (месяц, сезон, год и др.), единиц.
    '''
    
    result = []

    for gpmin_list in gp:
        gp_min = min(gpmin_list)  # Находим минимальное значение
        calculated_value = round(gp_min * GS * t, 1)
        result.append(calculated_value)

    return result

gp = calculation_gp(
        DGp, Ts, Tdp, vp
        )
t = 30
GS = 3.0

# Вызов функции
out = calculation_BBCqp3(
        gp, t, GS
        )
print(out) 

[270.0, 270.0, 180.0]


In [16]:
# Определяем список туристических объектов (!!! Все три объекта ПЛОЩАДНЫЕ !!!)
tourist_features = ['Объект 1', 'Объект 2', 'Объект 3']

# Определяем коэффициенты возвращения туристов на объекты
Rf_list = calculation_Rf([8, 7, 24], [5, 7, 12])
print(f'Коэффициенты возвращения туристов на объекты: {Rf_list}, единиц времени')

# Определяем базовую рекреационную емкость для туристских объектов ЕСЛИ ВСЕ ОНИ ПЛОЩАДНЫЕ 
# !!! пока логика разделения на линейные и площадные объекты на реализована и 
# все объекты должна задаваться одним типом: линейным или площадным !!!
BBCqs_list = calculation_BBCqs([100.2, 50.0, 25.1], 
                                [30.5, 12.0, 25.0], 
                                Rf_list, 
                                30)
print(f'Базовые рекреационные емкости: {BBCqs_list}, человек в единицу времени')

# Определяем потенциальную рекреационную емкость для каждого объекта
PCCq_list = calculation_PCCq(BBCqs_list, [[0.4, 0.5], [0.5, 2, 0.3], [0.8]])   
print(f'Потенциальные рекркационные емкости: {PCCq_list}, человек в единицу времени')

# Определяем коэффициенты управленческой емкости для каждого объекта
MC_list = [0.8, 0.5, 0.75]

# Вычисляем список придельно допустимых туристских емкостей объектов
RCCq_list = calculation_RCCq(PCCq_list, MC_list)
print(f'Придельно допустимые туристские емкости: {RCCq_list}, человек в единицу времени')

# Пусть у нас m туристических объекта c соответствующими предельно допустимыми рекреационными емкостями, 
# тогда предельно допустимая рекреационная емкость особо охраняемой природной территории RCCоопт будет равна   
RCCoopt_int = calculation_RCCoopt(tourist_features, RCCq_list)
print(f'Предельно допустимая рекреационная емкость ООПТ: {RCCoopt_int}, человек в единицу времени')

Коэффициенты возвращения туристов на объекты: [1.6, 1.0, 2.0], единиц времени
Базовые рекреационные емкости: [157, 125, 60], человек в единицу времени
Потенциальные рекркационные емкости: [31, 37, 48], человек в единицу времени
Придельно допустимые туристские емкости: [24, 18, 36], человек в единицу времени
Предельно допустимая рекреационная емкость ООПТ: ({'Объект 1': 24, 'Объект 2': 18, 'Объект 3': 36}, 78), человек в единицу времени


In [17]:
# Определяем список туристических объектов (!!! Все три объекта ЛИНЕЙНЫЕ !!!)
tourist_features = ['Объект 1', 'Объект 2', 'Объект 3']

# Определяем коэффициенты возвращения туристов на объекты
Rf_list = calculation_Rf([8, 7, 24], [5, 7, 12])
print(f'Коэффициенты возвращения туристов на объекты: {Rf_list}, единиц времени')

# Определяем базовую рекреационную емкость для туристских объектов для однодневных туристских маршрутов 
# и многодневных туристских маршрутов с небольшой протяженностью или несколькими входами 
# без ограничения времени посещения туристского маршрута, т.е. ЕСЛИ ВСЕ ОНИ ЛИНЕЙНЫЕ 
# !!! пока логика разделения на линейные и площадные объекты на реализована и 
# все объекты должна задаваться одним типом: линейным или площадным !!!
BBCqp1_list = calculation_BBCqp1([[20, 25], [30], [30, 20, 50]], 
                                [[5, 10], [10], [10, 5, 20]], 
                                8.0, 
                                [[4, 3], [3], [3, 4, 3]],
                                30,
                                3.1
                                )
print(f'Базовые рекреационные емкости линейных объектов BBCqp1: {BBCqp1_list}, человек в единицу времени')

print('- ' * 18)
# Максимальное количество групп на маршрутах
gp = calculation_gp([[5, 10], [10], [10, 5, 20]],
                    8.0,
                    [[4, 3], [3], [3, 4, 3]],
                    vp = [[4.0, 4.0], [5.0], [4.0, 4.0, 6.0]]
                    )
print(f'Максимальное количество групп на маршрутах: {gp}, единиц')

# Базовая рекреационная емкость для однодневных и многодневных туристских маршрутов, 
# время доступности которых строго фиксировано (BCC qp2) (например, в случае закрытия для посетителей 
# входа и выхода с туристского маршрута или в целом с участка особо охраняемой природной территории 
# в четко установленные часы), рассчитывается по формуле:
BBCqp2_list = calculation_BBCqp2(
        [[5, 6], [7], [7, 5, 12]], 
         t,
         GS
        )
print(f'Базовые рекреационные емкости линейных объектов BBCqp2: {BBCqp2_list}, человек в единицу времени')
print('- ' * 18)

# Базовая рекреационная емкость для автономных многодневных туристских маршрутов (BBCqp3) рассчитывается по формуле:
BBCqp3_list = calculation_BBCqp3(
        gp,
        t,
        GS
        )
print(f'Базовые рекреационные емкости линейных объектов BBCqp3: {BBCqp3_list}, человек в единицу времени')
print('- ' * 18)

# Определяем потенциальную рекреационную емкость для каждого объекта
PCCq_list = calculation_PCCq(BBCqp1_list, [[0.4, 0.5], [0.5, 2, 0.3], [0.8]])   
print(f'Потенциальные рекркационные емкости: {PCCq_list}, человек в единицу времени')

# Определяем коэффициенты управленческой емкости для каждого объекта
MC_list = [0.8, 0.5, 0.75]

# Вычисляем список придельно допустимых туристских емкостей объектов
RCCq_list = calculation_RCCq(PCCq_list, MC_list)
print(f'Придельно допустимые туристские емкости: {RCCq_list}, человек в единицу времени')

# Пусть у нас m туристических объекта c соответствующими предельно допустимыми рекреационными емкостями, 
# тогда предельно допустимая рекреационная емкость особо охраняемой природной территории RCCоопт будет равна   
RCCoopt_int = calculation_RCCoopt(tourist_features, RCCq_list)
print(f'Предельно допустимая рекреационная емкость ООПТ: {RCCoopt_int}, человек в единицу времени')

Коэффициенты возвращения туристов на объекты: [1.6, 1.0, 2.0], единиц времени
Базовые рекреационные емкости линейных объектов BBCqp1: [682.0, 744.0, 702.7], человек в единицу времени
- - - - - - - - - - - - - - - - - - 
Максимальное количество групп на маршрутах: [[4, 3], [3], [3, 4, 2]], единиц
Базовые рекреационные емкости линейных объектов BBCqp2: [495.0, 630.0, 720.0], человек в единицу времени
- - - - - - - - - - - - - - - - - - 
Базовые рекреационные емкости линейных объектов BBCqp3: [270.0, 270.0, 180.0], человек в единицу времени
- - - - - - - - - - - - - - - - - - 
Потенциальные рекркационные емкости: [136, 223, 562], человек в единицу времени
Придельно допустимые туристские емкости: [108, 111, 421], человек в единицу времени
Предельно допустимая рекреационная емкость ООПТ: ({'Объект 1': 108, 'Объект 2': 111, 'Объект 3': 421}, 640), человек в единицу времени


In [ ]:
# # Подключение к базе данных PostgreSQL
# def create_connection():
#     conn = psycopg2.connect(
#         dbname='turism',  
#         user='apple',        
#         password='admin',   
#         host='localhost',            
#         port='5432'                
#     )
#     return conn

In [ ]:
# # Создание таблицы в базе данных
# def create_table():
#     conn = create_connection()
#     cursor = conn.cursor()
#     create_table_query = """
#     CREATE TABLE IF NOT EXISTS geo_features (
#         id SERIAL PRIMARY KEY,
#         my_map TEXT,
#         place TEXT NOT NULL,
#         wiki_tags JSONB NOT NULL,
#         name_layer TEXT,
#         object_name TEXT,
#         geometry_type TEXT,
#         area FLOAT,
#         length FLOAT
#     );
#     """
#     cursor.execute(create_table_query)
#     conn.commit()
#     cursor.close()
#     conn.close()
#     print("Таблица geo_features успешно создана")

In [ ]:
# # Вставка данных в таблицу

# def insert_into_table(data):
#     conn = create_connection()
#     cursor = conn.cursor()
#     insert_query = """
#     INSERT INTO geo_features (my_map, place, wiki_tags, name_layer, object_name, geometry_type, area, length)
#     VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
#     """
#     # Преобразуем данные перед вставкой
#     data_to_insert = [
#         (
#             my_map,  # Здесь my_map уже строка
#             place,
#             json.dumps(wiki_tags),  # Преобразование словаря в строку JSON (сериализация)
#             name_layer,
#             object_name,
#             geometry_type,
#             area,
#             length
#         ) for (my_map, place, wiki_tags, name_layer, object_name, geometry_type, area, length) in data
#     ]
    
#     cursor.executemany(insert_query, data_to_insert)
#     conn.commit()
#     cursor.close()
#     conn.close()
# # 

In [ ]:
# def add_features_to_layers(
#         my_map, 
#         place, 
#         wiki_tags, 
#         name_layer_points=None,
#         color_points=f'rgb({45}, {210}, {100})',
#         name_layer_lines=None,
#         color_lines=f'rgb({210}, {45}, {45})',
#         name_layer_polygons=None,
#         fillcolor_polygons=f'rgb({50}, {170}, {235})',
#         color_polygons=f'rgb({50}, {120}, {235})'
#     ):
    
#     '''
#     Добавление слоев на карту по wiki-тегам с группировкой их по трем группам:
#     layer_points - точечные слои
#     layer_lines - линейные слои
#     layer_polygons - полигональные слои
#     my_map - карта
#     place - административное название области, например, "Камчатский край, Россия"
#     wiki_tags - теги рассматриваемых объектов (согласно https://wiki.openstreetmap.org/wiki/)
#     '''

#     # Инициализация слоев, только если имя слоя задано
#     layer_points = folium.FeatureGroup(name=name_layer_points) if name_layer_points else None
#     layer_lines = folium.FeatureGroup(name=name_layer_lines) if name_layer_lines else None
#     layer_polygons = folium.FeatureGroup(name=name_layer_polygons) if name_layer_polygons else None

#     # Добавляем слои на карту, только если они инициализированы
#     if layer_points:
#         layer_points.add_to(my_map)
#     if layer_lines:
#         layer_lines.add_to(my_map)
#     if layer_polygons:
#         layer_polygons.add_to(my_map)

#     # / / / / / / / / / /
#     # Список для записи в базу данных
#     data_to_insert = []

#     for row in fetch_features_with_generator(place, wiki_tags):
#         geo = row['geometry']
#         object_name = row.get('name', 'Unknown')
#         geometry_type = geo.geom_type

#         # Рассчитываем площадь или длину, если применимо
#         area = None
#         length = None
#         if geometry_type in ['Polygon', 'MultiPolygon']:
#             project = partial(
#                 pyproj.transform,
#                 pyproj.Proj(init="epsg:4326"),
#                 pyproj.Proj(proj="aea", lat_1=geo.bounds[1], lat_2=geo.bounds[3])
#             )
#             area = transform(project, shape(geo)).area
#         elif geometry_type in ['LineString', 'MultiLineString']:
#             length = geo.length
#         # / / / / / / / / / / 

#     for row in fetch_features_with_generator(place, wiki_tags):
        
#         popup_content = f"<b>Теги:</b>{''.join([f'<li>{k}: {row.get(k)}</li>' for k in wiki_tags.keys() if row.get(k)])}<br>"

#         if row.geometry.geom_type in ['Point', 'MultiPoint'] and layer_points:
#             folium.GeoJson(
#                 row.geometry,
#                 popup=folium.Popup(popup_content, max_width=300),
#                 style_function=lambda x: {'color': color_points, 'weight': 2}
#             ).add_to(layer_points)

#         elif row.geometry.geom_type in ['LineString', 'MultiLineString'] and layer_lines:
#             folium.GeoJson(
#                 row.geometry,
#                 popup=folium.Popup(popup_content, max_width=300),
#                 style_function=lambda x: {'color': color_lines, 'weight': 2}
#             ).add_to(layer_lines)

#         elif row.geometry.geom_type in ['Polygon', 'MultiPolygon'] and layer_polygons:
#             folium.GeoJson(
#                 row.geometry,
#                 popup=folium.Popup(popup_content, max_width=300),
#                 style_function=lambda x: {'fillColor': fillcolor_polygons, 'color': color_polygons, 'weight': 2, 'fillOpacity': 0.5}
#             ).add_to(layer_polygons)

#         # / / / / / / /
#         # Подготавливаем данные для записи в таблицу
#         data_to_insert.append((
#             str(my_map),    # Преобразуем карту в строку для хранения.
#             place,
#             wiki_tags,
#             name_layer_points or name_layer_lines or name_layer_polygons,  # Имя слоя
#             object_name,
#             geometry_type,
#             area,
#             length
#         ))

#     # Записываем данные в таблицу
#     if data_to_insert:
#         insert_into_table(data_to_insert)
    
#     # / / / / / / / / / / /

#     return my_map
